In [1]:
!pip install -e .. > /dev/null 2>&1

In [2]:
import os
import json
import astrohack

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [20, 20]

In [3]:
def build_folder_structure(paths):
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path) 

base_name = 'ea25_cal_small_'

datafolder = 'data'
resultsfolder = 'results'

# Build folder structure & download 
build_folder_structure(['data', 'results'])

graphviper.utils.data.download('ea25_cal_small_before_fixed.split.ms', folder=datafolder)
graphviper.utils.data.download('ea25_cal_small_after_fixed.split.ms', folder=datafolder)

graphviper.utils.data.download(file='extract_holog_verification.json')
graphviper.utils.data.download(file='holog_numerical_verification.json')

[2024-02-05 13:31:11,530]     INFO   astrohack:  File exists: data/ea25_cal_small_before_fixed.split.ms 
[2024-02-05 13:31:11,530]     INFO   astrohack:  File exists: data/ea25_cal_small_after_fixed.split.ms 
[2024-02-05 13:31:11,532]     INFO   astrohack:  File exists: extract_holog_verification.json 
[2024-02-05 13:31:11,533]     INFO   astrohack:  File exists: holog_numerical_verification.json 


In [4]:
from graphviper.dask.client import local_client

log_params = {
    'log_level':'DEBUG',
    'log_to_term': True
}

worker_log_params = {
    'log_level':'DEBUG',
    'log_file': None,
    'log_to_term': True,
    'log_to_file': False
}

client = local_client(
    cores=2, 
    memory_limit='8GB', 
    log_params=log_params,
    worker_log_params=worker_log_params
)

[2024-02-05 13:31:11,602]     INFO  graphviper:  Checking parameter values for client.local_client 
[2024-02-05 13:31:11,603]     INFO      logger:  /export/home/ajax/jhoskins/Development/graphviper-logger/ 
[2024-02-05 13:31:11,603]     INFO  graphviper:  Searching /export/home/ajax/jhoskins/Development/graphviper-logger/ for configuration file, please wait ... 
[2024-02-05 13:31:11,640]  WARNING  graphviper:  It is recommended that the local cache directory be set using the dask_local_dir parameter. 


/export/home/ajax/jhoskins/.conda/envs/holo-3.8/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40525 instead
  warnings.warn(


[2024-02-05 13:31:12,859]    DEBUG    worker_0:  Logger created on worker Worker-d845e83d-966c-4bc3-8e85-79fe8ffb1473,*,tcp://127.0.0.1:32789
[2024-02-05 13:31:12,859]    DEBUG    worker_1:  Logger created on worker Worker-43545f83-5c7c-4a52-b565-80d077e7ac94,*,tcp://127.0.0.1:42001
[2024-02-05 13:31:12,860]     INFO  graphviper:  Created client <MenrvaClient: 'tcp://127.0.0.1:39827' processes=2 threads=2, memory=14.90 GiB> 


In [5]:
def test_holog_obs_dictionary(holog_obs_dict):
    import os
    import json
    import copy
    
    import numpy as np

    with open(".holog_obs_dict.json") as json_file:
        holog_obj = json.load(json_file)
                          
    assert json.loads(holog_obj) == holog_obs_dict, "Error: holog_obs_descrition dictionary has changes unexpectedly."
    
def test_holog_diagnostics(json_data, tolerance=1e-7):
    import json
    
    with open("holog_numerical_verification.json") as file:
        reference_dict = json.load(file)
        
    cell_size = reference_dict["vla"]['cell_size'][1]
    grid_size = float(reference_dict["vla"]['grid_size'][1])
    
    json_data['cell_size'] = np.abs(float(json_data['cell_size']))
    
    cell_size = np.abs(float(cell_size))
    
    assert relative_difference(json_data['cell_size'], cell_size) < tolerance, "Unexpected change in cell_size occured."
    assert relative_difference(int(np.sqrt((json_data['n_pix']))), grid_size) < tolerance, "Unexpected change in grid_size occured."

    
def test_center_pixel(file, antenna, ddi, reference_center_pixels, number_of_digits=7):
    from astrohack.dio import open_image
    
    mds = open_image(file)[antenna][ddi]
    
    aperture_shape = mds.APERTURE.values.shape[-2], mds.APERTURE.values.shape[-1]
    beam_shape = mds.BEAM.values.shape[-2], mds.BEAM.values.shape[-1]    
    
    aperture_center_pixels = np.squeeze(mds.APERTURE.values[..., aperture_shape[0]//2, aperture_shape[1]//2])
    beam_center_pixels = np.squeeze(mds.BEAM.values[..., beam_shape[0]//2, beam_shape[1]//2])
    
    aperture_ref = list(map(complex, reference_center_pixels['aperture']))
    beam_ref = list(map(complex, reference_center_pixels['beam']))

    
    
    for i in range(len(aperture_ref)):        
        assert relative_difference(
            aperture_ref[i].real, 
            aperture_center_pixels[i].real
        ) < 1.5e-6, "There has been a shift in aperture center pixel value(s)"
        
        assert relative_difference(
            beam_ref[i].real, 
            beam_center_pixels[i].real
        ) < 1.5e-6, "There has been a shift in beam center pixel value(s)"
                
        assert relative_difference(
            aperture_ref[i].imag, 
            aperture_center_pixels[i].imag
        ) < 1.5e-6, "There has been a shift in aperture center pixel value(s)"
        assert relative_difference(
            beam_ref[i].imag, 
            beam_center_pixels[i].imag
        ) < 1e-6, "There has been a shift in beam center pixel value(s)"
        
              
def get_center_pixel(file, antenna, ddi):
    from astrohack.dio import open_image
    
    mds = open_image(file)[antenna][ddi]
    
    aperture_shape = mds.APERTURE.values.shape[-2], mds.APERTURE.values.shape[-1]
    beam_shape = mds.BEAM.values.shape[-2], mds.BEAM.values.shape[-1]    
    
    aperture_center_pixels = mds.APERTURE.values[..., aperture_shape[0]//2, aperture_shape[1]//2]
    beam_center_pixels = mds.BEAM.values[..., beam_shape[0]//2, beam_shape[1]//2]
    
    return np.squeeze(aperture_center_pixels), np.squeeze(beam_center_pixels)

def compare_float_values(result, reference, decimals):
    result_list = list(map(str, str(result)))
    reference_list = list(map(str, str(reference)))
    
    return result_list[:decimals] == reference_list[:decimals]

def relative_difference(result, expected):
    return 2*np.abs(result - expected)/(abs(result) + abs(expected))

In [6]:
from astrohack.extract_pointing import extract_pointing

before_ms = datafolder+'/'+ base_name + 'before_fixed.split.ms'
after_ms =  datafolder+'/'+ base_name + 'after_fixed.split.ms'

before_point = resultsfolder+'/before.split.point.zarr'
after_point = resultsfolder+'/after.split.point.zarr'

point_mds = extract_pointing(
      ms_name=before_ms,
      point_name=before_point,
      parallel=False,
      overwrite=True
)

holog_mds = extract_pointing(
      ms_name=after_ms,
      point_name=after_point,
      parallel=False,
      overwrite=True
)

[2024-02-05 13:31:12,904]     INFO   astrohack:  Checking parameter values for extract_pointing.extract_pointing 
[2024-02-05 13:31:12,906]     INFO   astrohack:  /export/home/ajax/jhoskins/Development/astrohack-development/ 
[2024-02-05 13:31:12,907]     INFO   astrohack:  Searching /export/home/ajax/jhoskins/Development/astrohack-development/src/astrohack/config/ for configuration file, please wait ... 
[2024-02-05 13:31:12,912]  WARNING   astrohack:  results/before.split.point.zarr will be overwritten. 
Successful readonly open of usernoread-locked table data/ea25_cal_small_before_fixed.split.ms/ANTENNA: 8 columns, 3 rows
[2024-02-05 13:31:17,600]     INFO   astrohack:  Finished processing 
[2024-02-05 13:31:17,643]     INFO   astrohack:  Checking parameter values for extract_pointing.extract_pointing 
[2024-02-05 13:31:17,644]     INFO   astrohack:  /export/home/ajax/jhoskins/Development/astrohack-development/ 
[2024-02-05 13:31:17,645]     INFO   astrohack:  Searching /export/home

In [29]:
from astrohack.extract_holog import extract_holog
import numpy as np

before_holog = resultsfolder+'/before.split.holog.zarr'
after_holog = resultsfolder+'/after.split.holog.zarr'

with open("extract_holog_verification.json") as file:
    json_dict = json.load(file)
    
holog_obs_dict = json_dict["vla"]

holg_mds = extract_holog(
    ms_name=after_ms,
    point_name=after_point,
    holog_name=after_holog,
    ddi=[0],
    #baseline_average_distance=50.1,
    #baseline_average_nearest=1,
    data_column='CORRECTED_DATA',
    parallel=False,
    overwrite=True
)

#test_holog_obs_dictionary(holog_obs_dict["after"])


with open(resultsfolder+'/after.split.holog.zarr/.holog_attr') as attr_file:
    holog_attr = json.load(attr_file)
    
#test_holog_diagnostics(
#    json_data=holog_attr,
#    tolerance=5e-5
#)


[2024-02-05 14:00:40,612]     INFO   astrohack:  Checking parameter values for extract_holog.extract_holog 
[2024-02-05 14:00:40,613]     INFO   astrohack:  /export/home/ajax/jhoskins/Development/astrohack-development/ 
[2024-02-05 14:00:40,615]     INFO   astrohack:  Searching /export/home/ajax/jhoskins/Development/astrohack-development/src/astrohack/config/ for configuration file, please wait ... 
[2024-02-05 14:00:40,619]  WARNING   astrohack:  results/after.split.holog.zarr will be overwritten. 
Successful readonly open of usernoread-locked table data/ea25_cal_small_after_fixed.split.ms/DATA_DESCRIPTION: 3 columns, 2 rows
Successful readonly open of usernoread-locked table data/ea25_cal_small_after_fixed.split.ms/ANTENNA: 8 columns, 3 rows
Successful readonly open of usernoread-locked table data/ea25_cal_small_after_fixed.split.ms: 23 columns, 36580 rows
[2024-02-05 14:00:40,680]     INFO   astrohack:              ea04       ea06        ea25
ea04    0.000000  195.96687   45.062448


In [31]:
list(holog_mds.keys())

['point_meta_ds', 'ant_ea04', 'ant_ea25', 'ant_ea06']

In [9]:
import numpy as np
from astrohack import holog

with open("holog_numerical_verification.json") as file:
    reference_dict = json.load(file)

grid_interpolation_mode = 'linear' #'nearest' 'linear' 'cubic'
chan_average = True
scan_average = True

holog(
    holog_name=before_holog,
    image_name="results/before.split.image.zarr",
    padding_factor=50, 
    grid_interpolation_mode=grid_interpolation_mode,
    chan_average = chan_average,
    scan_average = scan_average,
    overwrite=True,
    phase_fit=True,
    apply_mask=True,
    to_stokes=True,
    parallel=True
)

test_center_pixel(
    file="results/before.split.image.zarr", 
    antenna="ant_ea25", 
    ddi="ddi_0", 
    reference_center_pixels=reference_dict["vla"]["pixels"]["before"]
)

image_mds = holog(
    holog_name=after_holog,
    image_name="results/after.split.image.zarr",
    padding_factor=50, 
    grid_interpolation_mode=grid_interpolation_mode,
    chan_average = chan_average,
    scan_average = scan_average,
    overwrite=True,
    phase_fit=True,
    apply_mask=True,
    to_stokes=True,
    parallel=True
)

test_center_pixel(
    file="results/after.split.image.zarr", 
    antenna="ant_ea25", 
    ddi="ddi_0", 
    reference_center_pixels=reference_dict["vla"]["pixels"]["after"]
)

[2024-02-05 13:31:55,124]     INFO   astrohack:  Checking parameter values for holog.holog 
[2024-02-05 13:31:55,125]     INFO   astrohack:  /export/home/ajax/jhoskins/Development/astrohack-development/ 
[2024-02-05 13:31:55,127]     INFO   astrohack:  Searching /export/home/ajax/jhoskins/Development/astrohack-development/src/astrohack/config/ for configuration file, please wait ... 
[2024-02-05 13:31:55,136]  WARNING   astrohack:  results/before.split.image.zarr will be overwritten. 
[2024-02-05 13:31:55,151]     INFO   astrohack:  Cell size: [-0.00071948  0.00071948], Grid size [28 28] 
[2024-02-05 13:31:56,717]     INFO    worker_0:  Calculating aperture pattern ... 
[2024-02-05 13:31:56,717]     INFO    worker_0:  Calculating aperture illumination pattern ... 
[2024-02-05 13:31:56,776]     INFO    worker_1:  Calculating aperture pattern ... 
[2024-02-05 13:31:56,776]     INFO    worker_1:  Calculating aperture illumination pattern ... 
[2024-02-05 13:31:57,160]     INFO    worker_1

In [10]:
image_mds

{'ant_ea25': {'ddi_0': <xarray.Dataset>
  Dimensions:          (time: 1, chan: 1, pol: 4, u_prime: 552, v_prime: 552,
                        u: 627, v: 627, l: 29, m: 29, ddi: 1)
  Coordinates:
    * chan             (chan) float64 1.417e+10
    * ddi              (ddi) <U5 'ddi_0'
    * l                (l) float64 0.009507 0.008828 ... -0.008828 -0.009507
    * m                (m) float64 -0.009507 -0.008828 ... 0.008828 0.009507
    * pol              (pol) <U1 'I' 'Q' 'U' 'V'
    * u                (u) float64 735.1 732.8 730.4 ... -730.4 -732.8 -735.1
    * u_prime          (u_prime) float64 648.2 645.9 643.5 ... -643.5 -645.9
    * v                (v) float64 -735.1 -732.8 -730.4 ... 730.4 732.8 735.1
    * v_prime          (v_prime) float64 -648.2 -645.9 -643.5 ... 643.5 645.9
  Dimensions without coordinates: time
  Data variables:
      AMPLITUDE        (time, chan, pol, u_prime, v_prime) float64 dask.array<chunksize=(1, 1, 1, 276, 276), meta=np.ndarray>
      APERTURE     

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import scipy
import matplotlib.patches as patches

from scipy import constants

from astrohack.dio import open_image

before_image = resultsfolder+'/before.split.image.zarr'
after_image = resultsfolder+'/after.split.image.zarr'

plt.close('all')

chan = 0

ds = open_image("results/before.split.image.zarr")['ant_ea25']['ddi_0']

wavelength = scipy.constants.speed_of_light/ds.chan.values[chan]
l = ds.l.values
m = ds.m.values
u = ds.u.values*wavelength
v = ds.v.values*wavelength

print(u.min(),u.max())

plt.figure()
plt.imshow(np.abs(ds.BEAM[0,chan,0,:,:]),extent=[l.min(), l.max(), m.min(), m.max()])
plt.colorbar()
plt.show()

circle = patches.Circle((0,0), 12.5, fill=False, color='white', alpha=0.7, linewidth=2)

fig, ax =plt.subplots()
plt.imshow(np.abs(ds.APERTURE[0,chan,0,:,:]),extent=[u.min(), u.max(), v.min(), v.max()])

plt.show()

ds

In [ ]:
from astrohack.panel import panel

panel_model = 'rigid'

before_panel = panel(
    image_name=before_image,
    panel_name="results/before.split.panel.zarr",
    panel_model=panel_model,
    parallel=True,
    overwrite=True
)
after_panel = panel(
    image_name=after_image, 
    panel_name="results/after.split.panel.zarr",
    panel_model=panel_model,
    parallel=True,
    overwrite=True
)

In [ ]:
import numpy as np
from astrohack.dio import open_panel

def relative_difference(mean, expected):  
    return 2*np.abs(mean - expected)/(abs(mean) + abs(expected))

def verify_panel_shifts(
    panel_list=['3-4', '5-27', '5-37', '5-38'], 
    expected_shift=np.array([-100, 75, 0, 150]),
    ref_mean_shift = np.array([-77.8000519, 49.8347927, -0.0476941708, 100.268957]),
    antenna='ant_ea25',
    ddi='ddi_0'
):
    
    M_TO_MILS = 39370.1
    
    before_mds = open_panel('results/before.split.panel.zarr')
    after_mds = open_panel('results/after.split.panel.zarr')
    
    before_shift = before_mds[antenna][ddi].sel(labels=panel_list).PANEL_SCREWS.values*M_TO_MILS
    after_shift = after_mds[antenna][ddi].sel(labels=panel_list).PANEL_SCREWS.values*M_TO_MILS
    
    difference = after_shift - before_shift
    
    mean_shift = np.mean(difference, axis=1)
    
    delta_mean_shift = np.abs(mean_shift - expected_shift)
    delta_ref_shift = np.abs(ref_mean_shift - expected_shift)
        
    delta_shift = delta_mean_shift - delta_ref_shift  # New corrections - old corrections --> delta if delta < 0 ==> we improved.
    relative_shift = relative_difference(delta_mean_shift, delta_ref_shift)
       
    if np.any(relative_shift > 1e-6): 
        print("There were changes!")
        for i, delta in enumerate(delta_shift):
            if delta < 0:
                print("{panel}, improved by {delta} mils".format(panel=panel_list[i], delta=delta))
            else:
                print("{panel}, got worse by {delta} mils".format(panel=panel_list[i], delta=delta))

In [ ]:
verify_panel_shifts()